# Dashboard prototype
This is making a dashboard to visualise data using Dash and Plotly

In [1]:
!pip3 install jupyter-dash

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 101 kB 16.1 MB/s ta 0:00:01
     |████████████████████████████████| 10.2 MB 63.8 MB/s eta 0:00:01
     |████████████████████████████████| 15.2 MB 74.3 MB/s eta 0:00:01


In [118]:
import json
import plotly.express as px
import pandas as pd
from ast import literal_eval

In [4]:
df = pd.read_csv('../data/interim/CMS_2019_to_2022_ENGLISH.csv', index_col=0)

df.head()

/home/wojciech/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,shortTitle,text,teaser,shortTeaser,kicker,regions,keywords,keywordStrings,...,articles,isOpinion,geographicLocations,contentAssociations,mainContentImageLink,images,externalLinks,topStory,language,ts_lastModifiedDate
60278,46912921,"NASA spacecraft enters asteroid Bennu's orbit,...","NASA craft enters asteroid orbit, sets records",<p>NASA's unmanned spacecraft OSIRIS-REx enter...,The OSIRIS-REx spacecraft had arrived at the l...,NASA said its OSIRIS-REx spacecraft succesfull...,NaN,[],"[{'name': 'NASA'}, {'name': 'OSIRIS-REx'}, {'n...","['NASA', 'OSIRIS-REx', 'Bennu', 'asteroid']",...,"[{'name': 'After two-year journey, NASA Osiris...",False,[],"[{'id': 47376129}, {'id': 47376131}, {'id': 47...",{'description': None},[{'name': 'Sonde «Osiris Rex» nach zwei Jahren...,[{'name': 'DW Newsletter'}],False,NaN,2019-01-01 03:57:28.904000+00:00
60279,46911356,UK brings in boats to patrol English Channel a...,UK deploys boats amid Channel migration spike,<p>Britain will recall two overseas border pat...,The UK is withdrawing patrol ships from overse...,The UK is withdrawing patrol ships from overse...,NaN,[],"[{'name': 'English Channel'}, {'name': 'migrat...","['English Channel', 'migration', 'boats', 'ill...",...,[{'name': '40 migrants rescued from boats in E...,False,[],"[{'id': 47375206}, {'id': 47375208}, {'id': 47...",{'description': None},[{'name': 'Großbritanien | Zahl der Bootsflüch...,[],False,NaN,2019-01-01 06:11:50.527000+00:00
60280,46909694,"In Brazil, Jair Bolsonaro tries to unite the e...",Can Bolsonaro unite Brazil's motley right?,<p>Politics without ideologies and party cliqu...,Brazil is inaugurating President Jair Bolsonar...,"In his campaign, Jair Bolsonaro targeted appea...",NaN,[],"[{'name': 'Brazil'}, {'name': 'Jair Bolsonaro'...","['Brazil', 'Jair Bolsonaro', 'Chicago economic...",...,"[{'name': ""Bolsonaro's policies divide Afro-Br...",False,[],"[{'id': 47373913}, {'id': 47374152}, {'id': 47...",{'description': None},[{'name': 'Brasilien - Präsident Jair Bolsonar...,[{'name': ''}],False,NaN,2019-01-01 06:14:35.563000+00:00
60281,46912694,Tokyo: Man deliberately drives van into NYE cr...,Man drives car into Tokyo NYE crowd,<p>At least nine people were injured when a ma...,"A man with an ""intent to murder"" has driven a ...","A man with an ""intent to murder"" has driven a ...",NaN,[],"[{'name': 'Japan'}, {'name': 'Tokyo'}, {'name'...","['Japan', 'Tokyo', 'Harajuku', 'attack']",...,[],False,[],"[{'id': 47376009}, {'id': 47376011}, {'id': 47...",{'description': None},[{'name': 'Japan | Autoangriff in Tokyo'}],[],False,NaN,2019-01-01 08:26:11.599000+00:00
60282,46910092,Bangladesh opposition leader: International co...,International community 'shouldn't recognize n...,"<div class=""vjs-wrapper embed big ""><h2 aria-l...","In an exclusive interview with DW, Kamal Hossa...",Bangladeshi opposition leader Kamal Hossain sl...,NaN,[],"[{'name': 'Asia'}, {'name': 'Bangladesh'}, {'n...","['Asia', 'Bangladesh', 'elections', 'Kamal Hos...",...,"[{'name': ""Opinion: Bangladesh's missed chance...",False,[],"[{'id': 47374183}, {'id': 47374251}, {'id': 47...",{'description': None},"[{'name': 'Dr. Kamal Hossain '}, {'name': 'Pre...",[],False,NaN,2019-01-01 09:05:00.736000+00:00


In [10]:
keys_times = df[['ts_lastModifiedDate', 'keywordStrings']]
keys_times['keywordStrings'] = keys_times['keywordStrings'].apply(literal_eval)        #This ensures keywordStrings is an actual list, not just 'look-alike'
keys_times = keys_times.explode('keywordStrings')
keys_times.head()

/home/wojciech/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ts_lastModifiedDate,keywordStrings
60278,2019-01-01 03:57:28.904000+00:00,NASA
60278,2019-01-01 03:57:28.904000+00:00,OSIRIS-REx
60278,2019-01-01 03:57:28.904000+00:00,Bennu
60278,2019-01-01 03:57:28.904000+00:00,asteroid
60279,2019-01-01 06:11:50.527000+00:00,English Channel


In [11]:
top10 = keys_times['keywordStrings'].value_counts().nlargest(10)
print(top10.index)

Index(['Germany', 'coronavirus', 'COVID-19', 'China', 'Russia', 'Donald Trump',
       'Asia', 'US', 'EU', 'Bundesliga'],
      dtype='object')


In [13]:
keys_times['date']=pd.to_datetime(keys_times['ts_lastModifiedDate'])

keys_times["day"] = keys_times["date"].dt.day
keys_times["month"] = keys_times["date"].dt.month
keys_times["year"] = keys_times["date"].dt.year

grouped_keys_times = keys_times.groupby([keys_times.keywordStrings, keys_times.month, keys_times.year]).count().reset_index()
grouped_keys_times = grouped_keys_times.rename(columns={"date": "count"})
grouped_keys_times['yearmonth'] = pd.to_datetime(dict(year=grouped_keys_times.year, month=grouped_keys_times.month, day='01'))
grouped_keys_times.head()

/home/wojciech/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month] is missing

In [71]:
fig_df = grouped_keys_times.loc[grouped_keys_times['keywordStrings'].isin(top10.index)]
fig_df = fig_df.sort_values(by="yearmonth")

fig = px.line(fig_df, x='yearmonth', y="count", color="keywordStrings")
fig.show()

## Weeks series

In [112]:
keys_times = df[['ts_lastModifiedDate', 'keywordStrings']]
keys_times['keywordStrings'] = keys_times['keywordStrings'].apply(literal_eval)        #This ensures keywordStrings is an actual list, not just 'look-alike'
keys_times = keys_times.explode('keywordStrings')

keys_times['date'] = pd.to_datetime(keys_times['ts_lastModifiedDate'])
keys_times['yearweek'] = keys_times['date'].dt.strftime('%Y-%V')
grouped = keys_times.groupby(['keywordStrings', 'yearweek']).agg({'keywordStrings' : ['count']})
grouped = grouped.reset_index()
grouped = grouped.set_axis(['keywords', 'yearweek', 'count'], axis=1, inplace=False)
grouped.head()

/home/wojciech/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,keywords,yearweek,count
0,Refugees,2019-40,1
1,Refugees,2021-08,1
2,Refugees,2021-11,1
3,Refugees,2021-13,1
4,Refugees,2021-19,1


In [113]:
fig_df = grouped.loc[grouped['keywords'].isin(top10.index)]
fig_df = fig_df.sort_values(by="yearweek")

fig = px.line(fig_df, x='yearweek', y="count", color="keywords")
fig.show()

In [116]:
keys_times['date'] = pd.to_datetime(keys_times['ts_lastModifiedDate'])

keys_times.date

60278    2019-01-01 03:57:28.904000+00:00
60278    2019-01-01 03:57:28.904000+00:00
60278    2019-01-01 03:57:28.904000+00:00
60278    2019-01-01 03:57:28.904000+00:00
60279    2019-01-01 06:11:50.527000+00:00
                       ...               
172678   2021-12-31 20:38:11.201000+00:00
172678   2021-12-31 20:38:11.201000+00:00
150367   2022-01-01 02:35:51.098000+00:00
150367   2022-01-01 02:35:51.098000+00:00
150367   2022-01-01 02:35:51.098000+00:00
Name: date, Length: 176285, dtype: datetime64[ns, UTC]

## Using full JSON dataset and groupping into weeks

In [119]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

df = pd.DataFrame.from_dict(data)

df.head()

,id,name,shortTitle,text,teaser,shortTeaser,kicker,regions,keywords,keywordStrings,...,contentLinks,articles,isOpinion,geographicLocations,contentAssociations,mainContentImageLink,images,externalLinks,topStory,language
0,16489913,UN imposes sanctions on DRC rebels,UN sanctions DRC rebels,<p>\n\tThe UN Security Council has sanctioned ...,A UN Security Council sanctions committee has ...,,None,[],"[{'name': 'DRC'}, {'name': 'M23'}, {'name': 'F...","[DRC, M23, FDLR, Rwanda, Susan Rice, UN securi...",...,"[{'name': None}, {'name': 'Goma residents fear...",[{'name': 'Goma residents fear renewed conflic...,False,[],"[{'id': 30420921}, {'id': 30420923}, {'id': 30...",{'description': 'Kongo M23 Rebellen ziehen aus...,[{'name': 'Kongo M23 Rebellen ziehen aus beset...,[],False,None
1,16489912,Catholic Church abuse hotline goes cold,Disconnected,"<p>\n\tFor two and a half years, the counselin...",The Catholic Church in Germany has closed its ...,The Catholic Church has shut down a hotline fo...,None,[],"[{'name': 'telephone'}, {'name': 'hotline'}, {...","[telephone, hotline, helpline, Catholic Church...",...,"[{'name': None}, {'name': None}, {'name': None...",[],False,[],"[{'id': 30420944}, {'id': 30420945}, {'id': 30...",{'description': ''},"[{'name': 'Symbolbild Missbrauch Opfer'}, {'na...",[],False,None
2,16489903,"""Fiscal cliff"" tax hikes go into effect in the US","US goes over ""fiscal cliff""",<p>\n\tAs the clock struck midnight in Washing...,A series of tax hikes and spending cuts have g...,,None,[],"[{'name': 'fiscal cliff'}, {'name': 'Obama'}, ...","[fiscal cliff, Obama, congress, spending cuts,...",...,"[{'name': None}, {'name': '2012 defined by leg...",[{'name': '2012 defined by legislative gridloc...,False,[],"[{'id': 30420865}, {'id': 30420867}, {'id': 30...",{'description': ''},[{'name': 'Barack Obama PK Silvester Haushalts...,[],False,None
3,16490025,"Kim seeks reconciliation with South, economic ...",A rare New Year's address,"<p>\n\tKim Jong Un, who came to office just ov...",North Korea's leader has used a rare New Year’...,,None,[],"[{'name': 'Kim Jong Un'}, {'name': 'Kim Jong I...","[Kim Jong Un, Kim Jong Il, Kim Il Sung, lee my...",...,"[{'name': None}, {'name': 'South Korea condemn...",[{'name': 'South Korea condemns North Korea's ...,False,[],"[{'id': 30421237}, {'id': 30421239}, {'id': 30...",{'description': ''},[{'name': 'Nordkorea Neujahrsansprache von Kim...,[],False,None
4,16490029,Senate-approved fiscal deal faces House consid...,US House mulls fiscal deal,<p>\n\tLess than two hours after the US had of...,The US Senate has approved a bill to stop a se...,,None,[],"[{'name': 'fiscal cliff'}, {'name': 'US Senate...","[fiscal cliff, US Senate, Senate vote, fiscal ...",...,"[{'name': None}, {'name': 'Soft landing as US ...",[{'name': 'US reaches 'fiscal cliff' deal'}],False,[],"[{'id': 30421071}, {'id': 30421073}, {'id': 30...",{'description': ''},[{'name': 'USA Senat Fiskalklippe Verhandlunge...,[],False,None


In [127]:
import sys
# Add src folder to the path
sys.path.append('../src/')
from data.preprocess_keywords import make_cleaned_keywords_df
from data.make_datasets import get_data
# Specify wanted time range
start_date = '2019-01-01'
end_date = '2022-01-01'
# Path to data
data_file = '../data/raw/CMS_2010_to_June_2022_ENGLISH.json'
# Load and extract data within time range
df_subset = get_data(data_file, start_date, end_date)
# Cleans keywords and saves data as a dataframe
make_cleaned_keywords_df(df_subset, start_date, end_date)

Getting data...
Loading data DONE. Number of articles is 175659
Extracting data DONE. Number of articles from 2019-01-01 to 2022-01-01 is 33829
Cleaning step 1 out of 2 DONE. Number of unique keywords went from 32682 to 30228
Cleaning step 2 out of 2 DONE. Number of unique keywords went from 30228 to 27996
Finished. Data stored in ../data/interim/clean_keywords_2019-01-01_2022-01-01.json


: 

In [125]:
# Load data
filepath = '../data/interim/clean_keywords_' + start_date + '_' + end_date + '.json'
df_loaded = pd.read_json(filepath, orient ='split', compression = 'infer')

In [126]:
df_loaded.head()

,id,lastModifiedDate,keywordStrings,keywordStringsCleanAfterFuzz
0,46912921,2019-01-01T03:57:28.904Z,"[NASA, OSIRIS-REx, Bennu, asteroid]","[nasa, osiris-rex, bennu, asteroid]"
1,46911356,2019-01-01T06:11:50.527Z,"[English Channel, migration, boats, illegal im...","[english channel, migration, boats, illegal im..."
2,46909694,2019-01-01T06:14:35.563Z,"[Brazil, Jair Bolsonaro, Chicago economics, Ha...","[brazil, jair bolsonaro, chicago economics, ha..."
3,46912694,2019-01-01T08:26:11.599Z,"[Japan, Tokyo, Harajuku, attack]","[japan, tokyo, harajuku, attack]"
4,46910092,2019-01-01T09:05:00.736Z,"[Asia, Bangladesh, elections, Kamal Hossain, S...","[asia, bangladesh, elections, kamal hossain, s..."
